In [ ]:
%reload_ext autoreload
%autoreload 2
from importlib import reload

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import h5py
import matplotlib as mpl
from tqdm import tqdm


from holodeck import plot, detstats
import holodeck.anisotropy as anis
from holodeck.constants import YR, MSOL, MPC
import holodeck as holo
import healpy as hp

import hasasia.sim as hsim
import os


In [ ]:
SHAPE = None
NREALS = 500
# NREALS = 20
NFREQS = 40
NLOUDEST = 1000
BGL = 1

BUILD_ARRAYS = False
SAVEFIG = True
TOL=0.01
MAXBADS=5

NVARS = 3
# NVARS = 6

NPSRS = 40
NSKIES = 100
# NSKIES = 15

SHOW_GW=True
ALL_TARGETS = ['hard_time', 'gsmf_phi0', 'gsmf_mchar0_log10', 'mmb_mamp_log10', 'mmb_scatter_dex', 'hard_gamma_inner']

LMAX = 8
NSIDE = 8
PARVARS = [0,1,2]

In [ ]:
cmap_Blues = plot.truncate_colormap('Blues', 0.4, 1)
cmap_Greens = plot.truncate_colormap('Greens', 0.4, 1)
cmap_Oranges = plot.truncate_colormap('Oranges', 0.4, 1)

cmap_Greys = plot.truncate_colormap('Greys', 0.4, 0.9)

In [ ]:
def draw_anis_vars(ax, target, xx, yy_cl, params_cl, colors, parvars):
    ll=1
    handles = []
    for vv, var in enumerate(parvars):
        yy = yy_cl[vv][:,:,ll]/yy_cl[vv][:,:,0] # F,R
        pp = 68
        med, *conf = np.percentile(yy, (50, 50-pp/2, 50+pp/2), axis=-1)
        hh, = ax.plot(xx, med, label=params_cl[vv][target], color=colors[vv])
        handles.append(hh)
        ax.fill_between(xx, *conf, alpha=0.25, color=colors[vv])
    return handles
    

In [ ]:
BUILD_ARRAYS=True

xlabel = plot.LABEL_GW_FREQUENCY_YR
ylabel = r"$C_1/C_0$"
targets = [
    'gsmf_phi0', 'gsmf_mchar0_log10', 
      'mmb_mamp_log10',  'mmb_scatter_dex',
      'hard_time',  'hard_gamma_inner']
cmaps = [
    cmap_Greens, cmap_Greens,
    cmap_Oranges, cmap_Oranges,
    cmap_Blues, cmap_Blues
]
parvars = [0,1,2]

fig, axs = plot.figax_single(ncols=2, nrows=3, height=7,
                             sharey=True, sharex=True)
fig.text(0.55, 0.08, xlabel, va='center', ha='center')
fig.text(0.05, 0.55, ylabel, va='center', ha='center', rotation='vertical')

for ii, ax in enumerate(axs.flatten()):
    target = targets[ii]
    if BUILD_ARRAYS and target !='gsmf_phi0':
        yy_cl, xx_fobs, params_cl = detstats.build_anis_arrays(targets[ii], parvars=PARVARS, nloudest=NLOUDEST,
        nvars=NVARS, nreals=NREALS)
    else:
        yy_cl, xx_fobs, params_cl = detstats.get_anis_arrays(targets[ii], parvars=PARVARS, nloudest=NLOUDEST)

    xx = xx_fobs*YR

    colors = cmaps[ii](np.linspace(0,1,len(parvars)))
    ll=1
    handles = []
    for vv, var in enumerate(parvars):
        yy = yy_cl[vv][:,:,ll]/yy_cl[vv][:,:,0] # F,R
        pp = 68
        med, *conf = np.percentile(yy, (50, 50-pp/2, 50+pp/2), axis=-1)
        hh, = ax.plot(xx, med, label=params_cl[vv][target], color=colors[vv], zorder=6-vv)
        handles.append(hh)
        ax.fill_between(xx, *conf, alpha=0.25, color=colors[vv], zorder=3-vv)

    ax.legend(handles=handles, loc='lower right', ncols=3, title = plot.PARAM_KEYS[targets[ii]],
              labelspacing=0.1, borderpad=0.1, columnspacing=0.3, handletextpad=0.1, frameon=False, alignment='right')
    # ax.text(0.01,0.99, plot.PARAM_KEYS[targets[ii]], transform = ax.transAxes, va='top', ha='left')
fig.subplots_adjust(wspace=0, hspace=0)

savename = '/Users/emigardiner/GWs/holodeck/output/figures/bigplots/anisotropy'
savename += f"/C1C0_l{NLOUDEST}.png"
fig.savefig(savename, dpi=200, bbox_inches='tight')

In [ ]:
BUILD_ARRAYS=False

xlabel = plot.LABEL_GW_FREQUENCY_YR
xlabel2 = plot.LABEL_GW_FREQUENCY_NHZ
ylabel = r"$C_1/C_0$"
targets = [
    'gsmf_phi0', 'gsmf_mchar0_log10', 
      'mmb_mamp_log10',  'mmb_scatter_dex',
      'hard_time',  'hard_gamma_inner']
cmaps = [
    cmap_Greens, cmap_Greens,
    cmap_Oranges, cmap_Oranges,
    cmap_Blues, cmap_Blues
]
colors_grey = cmap_Greys(np.linspace(0,1,3))
parvars = [0,10,20]

fig, axs = plot.figax_single(ncols=2, nrows=3, height=7,
                             sharey=True, sharex=True)
fig.text(0.55, 0.08, xlabel, va='center', ha='center')
fig.text(0.55, 1.02, xlabel2, va='center', ha='center')
fig.text(0.05, 0.55, ylabel, va='center', ha='center', rotation='vertical')

for ii, ax in enumerate(axs.flatten()):
    target = targets[ii]
    if BUILD_ARRAYS:
        yy_cl, xx_fobs, params_cl = detstats.build_anis_arrays(targets[ii], parvars=PARVARS, nloudest=NLOUDEST,
        nvars=NVARS, nreals=NREALS)
    else:
        yy_cl, xx_fobs, params_cl = detstats.get_anis_arrays(targets[ii], parvars=PARVARS, nloudest=NLOUDEST)
    
    # add nloudest=10, for comparison
    yy_cl2, _, params_cl2 = detstats.get_anis_arrays(targets[ii], parvars=[0,10,20], nloudest=10)
    
    xx = xx_fobs*YR
    xx = xx_fobs*YR

    colors = cmaps[ii](np.linspace(0,1,len(parvars)))
    ll=1
    handles = []
    for vv, var in enumerate(parvars):
        yy = yy_cl[vv][:,:,ll]/yy_cl[vv][:,:,0] # F,R
        pp = 68
        med, *conf = np.percentile(yy, (50, 50-pp/2, 50+pp/2), axis=-1)
        hh, = ax.plot(xx, med, label=params_cl[vv][target], color=colors[vv], zorder=6-vv)
        handles.append(hh)
        ax.fill_between(xx, *conf, alpha=0.25, color=colors[vv], zorder=3-vv)

        y2 = yy_cl2[vv][:,:,ll]/yy_cl2[vv][:,:,0] # F,R
        pp = 68
        med2, *conf = np.percentile(y2, (50, 50-pp/2, 50+pp/2), axis=-1)
        ax.plot(xx, med2, label=params_cl[vv][target], color=colors_grey[vv], zorder=9-vv, linestyle=':')
        ax.fill_between(xx, *conf, alpha=0.25, color=colors_grey[vv], zorder=3-vv)

        difmax = np.max(np.abs(np.log10(med2) - np.log10(med)))
        maxes.append(difmax)

    ax.legend(handles=handles, loc='lower right', ncols=3, title = plot.PARAM_KEYS[targets[ii]],
              labelspacing=0.1, borderpad=0.1, columnspacing=0.3, handletextpad=0.1, frameon=False, alignment='right')
    ax.tick_params(axis='y', which='both', right=True, left=True, direction='in')
    # ax.tick_params(axis='x', which='both', top=True, direction='in')       
    # ax.text(0.01,0.99, plot.PARAM_KEYS[targets[ii]], transform = ax.transAxes, va='top', ha='left')

for ii, ax in enumerate(axs.flatten()[:2]):
    tw = ax.twiny()
    tw.grid(False)
    xlim = np.array(ax.get_xlim()) / YR * 1e9
    tw.set(xlim=xlim, xscale=ax.get_xscale())

    
fig.subplots_adjust(wspace=0, hspace=0)

print(np.sort(np.array(maxes)))
# savename = '/Users/emigardiner/GWs/holodeck/output/figures/bigplots/anisotropy'
# savename += f"/C1C0_l{NLOUDEST}.png"
# fig.savefig(savename, dpi=200, bbox_inches='tight')

In [ ]:
BUILD_ARRAYS=False

xlabel = plot.LABEL_GW_FREQUENCY_YR
xlabel2 = plot.LABEL_GW_FREQUENCY_NHZ
ylabel = r"$\sigma(C_1/C_0)$"
targets = [
    'gsmf_phi0', 'gsmf_mchar0_log10', 
      'mmb_mamp_log10',  'mmb_scatter_dex',
      'hard_time',  'hard_gamma_inner']
cmaps = [
    cmap_Greens, cmap_Greens,
    cmap_Oranges, cmap_Oranges,
    cmap_Blues, cmap_Blues
]
colors_grey = cmap_Greys(np.linspace(0,1,3))
parvars = [0,10,20]

fig, axs = plot.figax_single(ncols=2, nrows=3, height=7,
                             sharey=True, sharex=True)
fig.text(0.55, 0.08, xlabel, va='center', ha='center')
fig.text(0.55, 1.02, xlabel2, va='center', ha='center')
fig.text(0.05, 0.55, ylabel, va='center', ha='center', rotation='vertical')
maxes = []

for ii, ax in enumerate(axs.flatten()):
    target = targets[ii]
    if BUILD_ARRAYS:
        yy_cl, xx_fobs, params_cl = detstats.build_anis_arrays(targets[ii], parvars=PARVARS, nloudest=NLOUDEST,
        nvars=NVARS, nreals=NREALS)
    else:
        yy_cl, xx_fobs, params_cl = detstats.get_anis_arrays(targets[ii], parvars=PARVARS, nloudest=NLOUDEST)
    
    # using 10 loudest
    yy_cl2, _, params_cl2 = detstats.get_anis_arrays(targets[ii], parvars=[0,10,20], nloudest=10)
    
    xx = xx_fobs*YR

    colors = cmaps[ii](np.linspace(0,1,len(parvars)))
    ll=1
    handles = []
    for vv, var in enumerate(parvars):
        yy = np.std(yy_cl[vv][:,:,ll]/yy_cl[vv][:,:,0], axis=-1) # F,R
        # pp = 68
        # med, *conf = np.percentile(yy, (50, 50-pp/2, 50+pp/2), axis=-1)
        hh, = ax.plot(xx, yy, label=params_cl[vv][target], color=colors[vv], zorder=6-vv)
        handles.append(hh)
        # ax.fill_between(xx, *conf, alpha=0.25, color=colors[vv], zorder=3-vv)

        y2 = np.std(yy_cl2[vv][:,:,ll]/yy_cl2[vv][:,:,0], axis=-1) # F,R
        # pp = 68
        # med, *conf = np.percentile(yy, (50, 50-pp/2, 50+pp/2), axis=-1)
        ax.plot(xx, y2, label=params_cl[vv][target], color=colors_grey[vv], zorder=9-vv, linestyle=':')
        # ax.fill_between(xx, *conf, alpha=0.25, color=colors_grey[vv], zorder=3-vv)

        max_dif = np.max(np.abs(np.log10(yy)-np.log10(y2)))
        maxes.append(max_dif)

    ax.legend(handles=handles, loc='lower right', ncols=3, title = plot.PARAM_KEYS[targets[ii]],
              labelspacing=0.1, borderpad=0.1, columnspacing=0.3, handletextpad=0.1, frameon=False, alignment='right')
    ax.tick_params(axis='y', which='both', right=True, left=True, direction='in')
    # ax.tick_params(axis='x', which='both', top=True, direction='in')       
    # ax.text(0.01,0.99, plot.PARAM_KEYS[targets[ii]], transform = ax.transAxes, va='top', ha='left')

for ii, ax in enumerate(axs.flatten()[:2]):
    tw = ax.twiny()
    tw.grid(False)
    xlim = np.array(ax.get_xlim()) / YR * 1e9
    tw.set(xlim=xlim, xscale=ax.get_xscale())

    
fig.subplots_adjust(wspace=0, hspace=0)

savename = '/Users/emigardiner/GWs/holodeck/output/figures/bigplots/anisotropy'
savename += f"/sigmaC1C0_l{NLOUDEST}.png"
fig.savefig(savename, dpi=200, bbox_inches='tight')

print(np.sort(np.array(maxes)))

In [ ]:
BUILD_ARRAYS=False

xlabel = plot.LABEL_GW_FREQUENCY_YR
xlabel2 = plot.LABEL_GW_FREQUENCY_NHZ
ylabel = r"$C_1/C_0$"
targets = [
    'gsmf_phi0', 'gsmf_mchar0_log10', 
      'mmb_mamp_log10',  'mmb_scatter_dex',
      'hard_time',  'hard_gamma_inner']
cmaps = [
    cmap_Greens, cmap_Greens,
    cmap_Oranges, cmap_Oranges,
    cmap_Blues, cmap_Blues
]
parvars = [0,10,20]

fig, axs = plot.figax_single(ncols=2, nrows=3, height=7,
                             sharey=True, sharex=True)
fig.text(0.55, 0.08, xlabel, va='center', ha='center')
fig.text(0.55, 1.02, xlabel2, va='center', ha='center')
fig.text(0.05, 0.55, ylabel, va='center', ha='center', rotation='vertical')

for ii, ax in enumerate(axs.flatten()):
    target = targets[ii]
    if BUILD_ARRAYS:
        yy_cl, xx_fobs, params_cl = detstats.build_anis_arrays(targets[ii], parvars=PARVARS, nloudest=NLOUDEST,
        nvars=NVARS, nreals=NREALS)
    else:
        yy_cl, xx_fobs, params_cl = detstats.get_anis_arrays(targets[ii], parvars=PARVARS, nloudest=NLOUDEST)

    xx = xx_fobs*YR
    xx = xx_fobs*YR

    colors = cmaps[ii](np.linspace(0,1,len(parvars)))
    ll=1
    handles = []
    for vv, var in enumerate(parvars):
        yy = yy_cl[vv][:,:,ll]/yy_cl[vv][:,:,0] # F,R
        pp = 68
        med, *conf = np.percentile(yy, (50, 50-pp/2, 50+pp/2), axis=-1)
        hh, = ax.plot(xx, med, label=params_cl[vv][target], color=colors[vv], zorder=6-vv)
        handles.append(hh)
        ax.fill_between(xx, *conf, alpha=0.25, color=colors[vv], zorder=3-vv)

    # add nloudest=10, for comparison
    yy_cl, _, params_cl = detstats.get_anis_arrays(targets[ii], parvars=[0,10,20], nloudest=10)
    for vv, var in enumerate(parvars):
        yy = yy_cl[vv][:,:,ll]/yy_cl[vv][:,:,0] # F,R
        pp = 68
        med, *conf = np.percentile(yy, (50, 50-pp/2, 50+pp/2), axis=-1)
        ax.plot(xx, med, label=params_cl[vv][target], color=colors[vv], zorder=6-vv, linestyle=':')

    ax.legend(handles=handles, loc='lower right', ncols=3, title = plot.PARAM_KEYS[targets[ii]],
              labelspacing=0.1, borderpad=0.1, columnspacing=0.3, handletextpad=0.1, frameon=False, alignment='right')
    ax.tick_params(axis='y', which='both', right=True, left=True, direction='in')
    # ax.tick_params(axis='x', which='both', top=True, direction='in')       
    # ax.text(0.01,0.99, plot.PARAM_KEYS[targets[ii]], transform = ax.transAxes, va='top', ha='left')

for ii, ax in enumerate(axs.flatten()[:2]):
    tw = ax.twiny()
    tw.grid(False)
    xlim = np.array(ax.get_xlim()) / YR * 1e9
    tw.set(xlim=xlim, xscale=ax.get_xscale())

    
fig.subplots_adjust(wspace=0, hspace=0)

savename = '/Users/emigardiner/GWs/holodeck/output/figures/bigplots/anisotropy'
savename += f"/C1C0_l{NLOUDEST}.png"
fig.savefig(savename, dpi=200, bbox_inches='tight')

In [ ]:
BUILD_ARRAYS=False

xlabel = plot.LABEL_GW_FREQUENCY_YR
ylabel = r"$C_1/C_0$"
targets = [
    'gsmf_phi0', 'gsmf_mchar0_log10', 
      'mmb_mamp_log10',  'mmb_scatter_dex',
      'hard_time',  'hard_gamma_inner']
cmaps = [
    cmap_Greens, cmap_Greens,
    cmap_Oranges, cmap_Oranges,
    cmap_Blues, cmap_Blues
]
parvars = [0,10,20]

fig, axs = plot.figax_single(ncols=2, nrows=3, height=7,
                             sharey=True, sharex=True)
fig.text(0.55, 0.08, xlabel, va='center', ha='center')
fig.text(0.05, 0.55, ylabel, va='center', ha='center', rotation='vertical')

for ii, ax in enumerate(axs.flatten()):
    target = targets[ii]
    if BUILD_ARRAYS:
        yy_cl, xx_fobs, params_cl = detstats.build_anis_arrays(targets[ii], parvars=PARVARS, nloudest=NLOUDEST,
        nvars=NVARS, nreals=NREALS)
    else:
        yy_cl, xx_fobs, params_cl = detstats.get_anis_arrays(targets[ii], parvars=PARVARS, nloudest=NLOUDEST)

    xx = xx_fobs*YR
    xx = xx_fobs*YR

    colors = cmaps[ii](np.linspace(0,1,len(parvars)))
    ll=1
    handles = []
    for vv, var in enumerate(parvars):
        yy = yy_cl[vv][:,:,ll]/yy_cl[vv][:,:,0] # F,R
        pp = 68
        med, *conf = np.percentile(yy, (50, 50-pp/2, 50+pp/2), axis=-1)
        hh, = ax.plot(xx, med, label=params_cl[vv][target], color=colors[vv], zorder=6-vv)
        handles.append(hh)
        ax.fill_between(xx, *conf, alpha=0.25, color=colors[vv], zorder=3-vv)

    # add nloudest=10, for comparison
    yy_cl, _, params_cl = detstats.get_anis_arrays(targets[ii], parvars=[0,10,20], nloudest=10)
    for vv, var in enumerate(parvars):
        yy = yy_cl[vv][:,:,ll]/yy_cl[vv][:,:,0] # F,R
        pp = 68
        med, *conf = np.percentile(yy, (50, 50-pp/2, 50+pp/2), axis=-1)
        ax.plot(xx, med, label=params_cl[vv][target], color=colors[vv], zorder=6-vv, linestyle=':')

    ax.legend(handles=handles, loc='lower right', ncols=3, title = plot.PARAM_KEYS[targets[ii]],
              labelspacing=0.1, borderpad=0.1, columnspacing=0.3, handletextpad=0.1, frameon=False, alignment='right')
    ax.tick_params(axis='y', which='both', right=True, left=True, direction='in')
    ax.tick_params(axis='x', which='both', top=True, direction='in')       
    # ax.text(0.01,0.99, plot.PARAM_KEYS[targets[ii]], transform = ax.transAxes, va='top', ha='left')
fig.subplots_adjust(wspace=0, hspace=0)

savename = '/Users/emigardiner/GWs/holodeck/output/figures/bigplots/anisotropy'
savename += f"/C1C0_l{NLOUDEST}.png"
fig.savefig(savename, dpi=200, bbox_inches='tight')